In [5]:
import pandas as pd
import numpy as np

In [37]:
cerebras_df = pd.read_excel("data/testi_cerebras_20250608.xlsx")
mistral_df = pd.read_excel("data/testi_mistral_20250608.xlsx")
qwen_df = pd.read_excel("data/testi_qwen_20250608.xlsx")

In [38]:
cerebras_df["model"] = "llama-4-scout-17b-16e-instruct"
mistral_df["model"] = "mistral-large-latest"
qwen_df["model"] = "qwen2.5-72b-instruct"

In [39]:
all_df = pd.concat([cerebras_df, mistral_df, qwen_df], ignore_index=True)

all_df = all_df.sample(frac=1, random_state=42).drop_duplicates(subset='testo', keep='first').reset_index(drop=True)

In [40]:
all_df.head()

,edizione,sezione,titoletto,testo,input,model
0,2023Q2,corpo,L’occupazione continuerebbe a espandersi,Il saldo fra la quota di aziende che intendono...,Il saldo tra le aziende che vogliono aumentare...,qwen2.5-72b-instruct
1,2023Q3,corpo,Le aspettative di inflazione sono scese su tut...,Le aspettative sull’inflazione al consumo si s...,Le aspettative sull'inflazione al consumo si s...,qwen2.5-72b-instruct
2,2020Q3,sintesi,NaN,Secondo l’indagine condotta tra il 27 agosto e...,Ecco una possibile bozza precedente:\n\nDall'i...,llama-4-scout-17b-16e-instruct
3,2024Q3,corpo,Le aspettative di inflazione sono lievemente a...,"Rispetto alla scorsa rilevazione, l’inflazione...","Okay, quindi, rispetto all'ultima rilevazione,...",mistral-large-latest
4,2021Q2,corpo,…così come quelli sull’andamento corrente e at...,Le imprese si attendono anche una forte cresci...,Le imprese pensano che la domanda dei loro pro...,llama-4-scout-17b-16e-instruct


In [41]:
all_df.model.value_counts()

model
qwen2.5-72b-instruct              93
llama-4-scout-17b-16e-instruct    87
mistral-large-latest              83
Name: count, dtype: int64

In [42]:
all_df.to_excel("data/testi_randomized.xlsx", index=False)